In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
import shutil
import sys
import json

# TRAINING HYPER-PARAMETERS

In [5]:
params_df_list = []

# Define fixed seed for BIRCH
BIRCH_SEED = 42  # You can choose any constant value

# Generate random seeds for other algorithms
DYNAMIC_SEEDS = np.random.choice(1000, size=4, replace=False)

params_dict = {
    # "instrument": ["MSFT_M15", "AMZN_M15", "NVDA_M15", "TSLA_M15", "GOOGL_M15", "META_M15"],
    "instrument": ["AAPL_M15", "MSFT_M15"],
    "price_history_length": [24],
    "num_perceptually_important_points": [4],
    "num_clusters": [5, 6, 7, 8, 9],
    "clustering_algorithm": ["kmeans", "birch"],
    "train_period": [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "test_period": [1],
    "reverse_test": [True, False],
}

# Create separate dataframes for BIRCH and other algorithms
# BIRCH parameters
birch_params = params_dict.copy()
birch_params["clustering_algorithm"] = ["birch"]
birch_params["random_seed"] = [BIRCH_SEED]

# Other algorithms parameters
other_params = params_dict.copy()
other_params["clustering_algorithm"] = ["kmeans"]
other_params["random_seed"] = DYNAMIC_SEEDS

# Generate parameter combinations for BIRCH
birch_df = (
    pd.MultiIndex.from_product(birch_params.values(), names=birch_params.keys())
    .to_frame()
    .reset_index(drop=True)
)

# Generate parameter combinations for other algorithms
other_df = (
    pd.MultiIndex.from_product(other_params.values(), names=other_params.keys())
    .to_frame()
    .reset_index(drop=True)
)

# Combine both dataframes
params_concat_df = pd.concat([birch_df, other_df], ignore_index=True)
params_concat_df

,instrument,price_history_length,num_perceptually_important_points,num_clusters,clustering_algorithm,train_period,test_period,reverse_test,random_seed
0,AAPL_M15,24,4,5,birch,4,1,True,42
1,AAPL_M15,24,4,5,birch,4,1,False,42
2,AAPL_M15,24,4,5,birch,5,1,True,42
3,AAPL_M15,24,4,5,birch,5,1,False,42
4,AAPL_M15,24,4,5,birch,6,1,True,42
...,...,...,...,...,...,...,...,...,...
1095,MSFT_M15,24,4,9,kmeans,14,1,True,227
1096,MSFT_M15,24,4,9,kmeans,14,1,False,840
1097,MSFT_M15,24,4,9,kmeans,14,1,False,342
1098,MSFT_M15,24,4,9,kmeans,14,1,False,545


# TESTING BEST HYPER-PARAMETERS

In [ ]:
import pandas as pd

# Constants for repeated values
INSTRUMENTS = [
    'EUR_USD_M15', 'GBP_USD_M15', 'USD_JPY_M15', 'USD_CHF_M15', 
    'USD_CAD_M15', 'AUD_USD_M15', 'AUD_JPY_M15', 'AUD_CAD_M15',
    'EUR_GBP_M15', 'EUR_JPY_M15', 'GBP_CHF_M15', 'GBP_JPY_M15',
    'EUR_CHF_M15', 'AUD_NZD_M15', 'CAD_JPY_M15', 'NZD_USD_M15', 
    'EUR_CAD_M15'
]
NUM_PAIRS = len(INSTRUMENTS)

best_params = {
    "instrument": INSTRUMENTS,
    "num_clusters": [5] * NUM_PAIRS,  # All pairs use 5 clusters
    "clustering_algorithm": [
        'kmeans', 'kmeans', 'kmeans', 'birch',
        'kmeans', 'kmeans', 'birch', 'birch',
        'kmeans', 'kmeans', 'kmeans', 'birch',
        'birch', 'kmeans', 'birch', 'kmeans',
        'kmeans'
    ],
    "train_period": [
        4, 11, 11, 11,
        13, 11, 9, 6,
        5, 4, 7, 9,
        4, 10, 4, 8,
        7
    ],
    "test_period": [1] * NUM_PAIRS,
    "price_history_length": [24] * NUM_PAIRS,  # Keeping this constant parameter
    "num_perceptually_important_points": [4] * NUM_PAIRS,  # Keeping this constant parameter
    "reverse_test": [
        False, True, True, True,
        True, True, True, False,
        True, True, True, True,
        False, True, False, False,
        True
    ],
    "random_seed": [
        60, 233, 421, 42,
        231, 640, 42, 42,
        279, 886, 623, 42,
        42, 594, 42, 370,
        673
    ]
}

params_concat_df = pd.DataFrame(best_params)
params_concat_df

In [6]:
n_tasks, ncol = params_concat_df.shape
date_time = datetime.now().strftime("%Y-%m-%d_%H:%M")
job_name = f"ml_project_2_{date_time}"
job_dir = "/scratch/da2343/" + job_name
results_dir = os.path.join(job_dir, "results")
os.system("mkdir -p " + results_dir)
params_concat_df.to_csv(os.path.join(job_dir, "params.csv"), index=False)

print(f"created {n_tasks} tasks in {job_dir}")

run_one_contents = f"""#!/bin/bash
#SBATCH --array=0-{n_tasks-1}
#SBATCH --time=5:00:00
#SBATCH --mem=4GB
#SBATCH --cpus-per-task=1
#SBATCH --error={job_dir}/slurm-%A_%a.out
#SBATCH --output={job_dir}/slurm-%A_%a.out
#SBATCH --job-name={job_name}
cd {job_dir}
python run_one.py $SLURM_ARRAY_TASK_ID
"""
run_one_sh = os.path.join(job_dir, "run_one.sh")
with open(run_one_sh, "w") as run_one_f:
    run_one_f.write(run_one_contents)

run_orig_py = "demo_run_stock.py"
run_one_py = os.path.join(job_dir, "run_one.py")
shutil.copyfile(run_orig_py, run_one_py)
orig_dir = os.path.dirname(run_orig_py)
orig_results = os.path.join(orig_dir, "results")
os.system("mkdir -p " + orig_results)
orig_csv = os.path.join(orig_dir, "params.csv")
params_concat_df.to_csv(orig_csv, index=False)

msg = f"""created params CSV files and job scripts, test with
python {run_orig_py}
SLURM_ARRAY_TASK_ID=0 bash {run_one_sh}"""
print(msg)


created 1100 tasks in /scratch/da2343/ml_project_2_2025-01-16_20:36
created params CSV files and job scripts, test with
python demo_run_stock.py
SLURM_ARRAY_TASK_ID=0 bash /scratch/da2343/ml_project_2_2025-01-16_20:36/run_one.sh
